# ETL sobre 'user_reviews'

Importamos librerias 

In [1]:
from pandas import json_normalize
import pandas as pd 
import json 
import ast 

Guardamos la ruta de acceso  dentro de una variable 

In [2]:
path_reviews =r'C:\Users\Usuario\Desktop\proyecto steam\Dataset\australian_user_reviews.json'

Extraemos datos de "australian_user_reviews.json" , creamos un Dataframe del archivo original.\
Creamos una copia del Dataframe para sus transformaciones.


In [3]:

lista_reviews = []
with open(path_reviews, encoding='utf-8') as file:
    for line in file.readlines():
        lista_reviews.append(ast.literal_eval(line))

 
df_reviews = pd.DataFrame(lista_reviews)

df_reviews_clean = df_reviews.copy()

Controlamos las primeras 10 filas del dataframe

In [4]:
df_reviews_clean.head(10)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
5,Wackky,http://steamcommunity.com/id/Wackky,"[{'funny': '', 'posted': 'Posted May 5, 2014.'..."
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,"[{'funny': '1 person found this review funny',..."
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,"[{'funny': '', 'posted': 'Posted July 24.', 'l..."
8,76561198089393905,http://steamcommunity.com/profiles/76561198089...,"[{'funny': '5 people found this review funny',..."
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."


Borramos la columna user_url (No es util para el proyecto) 

In [5]:
df_reviews_clean = df_reviews_clean.drop('user_url', axis=1)

Desanidamos la columna 'reviews'

In [6]:
from pandas import json_normalize

df_expanded = df_reviews_clean.explode('reviews')


Reindexamos 

In [7]:
df_expanded.reset_index(drop=True, inplace=True)
#df_expanded.head(20) -- Controlamos resultado del codigo

Normalizamos los datos de la columna 'reviews'

In [8]:
df_reviews_normalized = json_normalize(df_expanded['reviews'])
#df_reviews_normalized.head(20) -- Controlamos resultado del codigo

 Combinamos con el DataFrame original y normalizado

In [9]:
df_final = pd.concat([df_expanded.drop(columns=['reviews']), df_reviews_normalized], axis=1)
#df_final.head(20) -- Controlamos resultado del codigo

Eliminamos las columnas 'funny' ,'last_edited', 'helpful' (No son utiles para el proyecto)

In [10]:
# Lista de columnas a eliminar
columns_to_drop = ['funny', 'last_edited', 'helpful']

# Eliminar las columnas especificadas
df_reviews_clean = df_final.drop(columns=columns_to_drop, errors='ignore')


Exportamos el archivo a .parquet 

In [11]:
df_reviews_clean.to_parquet(r'C:\Users\Usuario\Desktop\proyecto steam\Data Exportada\user_reviews_clean.parquet', index=False)